In [1]:
import sys
sys.path.append('..')

from pathlib import Path
import joblib
import pandas as pd

from src.data.preprocess import load_raw, split_features_target
from src.data.feature_engineering import apply_feature_engineering

PROJECT_ROOT = Path.cwd().resolve().parents[0]
MODELS_DIR = PROJECT_ROOT / "src" / "models"

EXPECTED_REMAINING_MONTHS = 12


In [2]:
clf = joblib.load(MODELS_DIR / "model.pkl")

df, DATA_DIR = load_raw()
df = apply_feature_engineering(df)
X, y, id_cols = split_features_target(df)

MODELS_DIR = DATA_DIR.parent / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)


In [3]:
y_proba = clf.predict_proba(X)[:, 1]

df_scores = df.copy()
df_scores["churn_proba"] = y_proba

if "MonthlyCharges" not in df_scores.columns:
    raise ValueError("MonthlyCharges column required for impact calculation.")

df_scores["revenue_at_risk_month"] = df_scores["churn_proba"] * df_scores["MonthlyCharges"]
df_scores["expected_loss_total"] = df_scores["revenue_at_risk_month"] * EXPECTED_REMAINING_MONTHS

df_scores.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TotalCharges,Churn,tenure_bucket,avg_monthly_total,is_month_to_month,has_fiber,multi_service,churn_proba,revenue_at_risk_month,expected_loss_total
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,29.85,No,0-12,29.85,1,0,0,0.684701,20.438329,245.259950
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,1889.50,No,25-48,55.573529,0,0,1,0.024523,1.396612,16.759350
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,108.15,Yes,0-12,54.075,1,0,1,0.352080,18.959493,227.513915
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,1840.75,No,25-48,40.905556,0,0,0,0.029276,1.238374,14.860490
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,151.65,Yes,0-12,75.825,1,1,1,0.654330,46.261107,555.133281


In [ ]:
segments = ["Contract", "tenure_bucket"]

seg_agg = (
    df_scores.groupby(segments, observed=False)
    .agg(
        n_customers=("churn_proba", "size"),
        avg_churn_proba=("churn_proba", "mean"),
        total_revenue_at_risk_month=("revenue_at_risk_month", "sum"),
        total_expected_loss=("expected_loss_total", "sum"),
    )
    .reset_index()
)

seg_agg.sort_values("total_expected_loss", ascending=False).head(10)


,Contract,tenure_bucket,n_customers,avg_churn_proba,total_revenue_at_risk_month,total_expected_loss
0,Month-to-month,0-12,1994,0.510908,68600.759958,823209.119490
2,Month-to-month,25-48,802,0.324195,22254.722096,267056.665151
1,Month-to-month,13-24,737,0.380183,22142.022600,265704.271197
3,Month-to-month,49-72,342,0.286429,8755.248932,105062.987180
8,One year,49-72,634,0.127163,7561.199447,90734.393364
7,One year,25-48,518,0.104281,4379.066849,52548.802185
13,Two year,49-72,1263,0.032325,3640.414038,43684.968453
6,One year,13-24,197,0.078153,989.246531,11870.958373
5,One year,0-12,124,0.081118,572.969463,6875.633560
12,Two year,25-48,274,0.025781,539.297165,6471.565978


In [5]:
from src.models.impact import expected_remaining_months

df_scores["horizon"] = df_scores.apply(expected_remaining_months, axis=1)
df_scores["expected_loss_total"] = (
    df_scores["churn_proba"] * df_scores["MonthlyCharges"] * df_scores["horizon"]
)
df_scores

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Churn,tenure_bucket,avg_monthly_total,is_month_to_month,has_fiber,multi_service,churn_proba,revenue_at_risk_month,expected_loss_total,horizon
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,0-12,29.85,1,0,0,0.684701,20.438329,122.629975,6
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,25-48,55.573529,0,0,1,0.024523,1.396612,1.396612,1
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Yes,0-12,54.075,1,0,1,0.352080,18.959493,113.756957,6
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,25-48,40.905556,0,0,0,0.029276,1.238374,1.238374,1
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Yes,0-12,75.825,1,1,1,0.654330,46.261107,277.566641,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,No,13-24,82.9375,0,0,1,0.074937,6.354688,6.354688,1
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,No,49-72,102.2625,0,1,1,0.062911,6.492437,6.492437,1
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,0-12,31.495455,1,0,0,0.226505,6.704549,40.227293,6
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,Yes,0-12,76.65,1,1,1,0.703026,52.305165,313.830991,6


In [6]:
segments = ["Contract", "tenure_bucket", "PaymentMethod"]

impact_by_segment = df_scores.groupby(segments, observed=False).agg(
    customers=("churn_proba", "size"),
    avg_proba=("churn_proba", "mean"),
    loss=("expected_loss_total", "sum"),
).sort_values("loss", ascending=False)
impact_by_segment

customers  avg_proba  \
Contract       tenure_bucket PaymentMethod                                     
Month-to-month 0-12          Electronic check                 954   0.644256   
               13-24         Electronic check                 355   0.493965   
               25-48         Electronic check                 376   0.416426   
               0-12          Mailed check                     641   0.366555   
                             Bank transfer (automatic)        208   0.436604   
                             Credit card (automatic)          191   0.410237   
               49-72         Electronic check                 165   0.358511   
               25-48         Bank transfer (automatic)        172   0.261006   
                             Credit card (automatic)          159   0.269222   
               13-24         Bank transfer (automatic)        120   0.306545   
                             Credit card (automatic)          121   0.306867   
                             Mailed check                     141   0.219297   
               49-72         Bank transfer (automatic)         89   0.210999   
                             Credit card (automatic)           72   0.248227   
               25-48         Mailed check                      95   0.165569   
One year       49-72         Electronic check                 184   0.181299   
                             Bank transfer (automatic)        194   0.114547   
                             Credit card (automatic)          196   0.110697   
               25-48         Electronic check                 117   0.165701   
Two year       49-72         Bank transfer (automatic)        479   0.030668   
One year       25-48         Credit card (automatic)          140   0.100453   
Two year       49-72         Credit card (automatic)          469   0.026736   
                             Electronic check                 138   0.081847   
One year       25-48         Bank transfer (automatic)        140   0.089109   
Month-to-month 49-72         Mailed check                      16   0.134583   
One year       0-12          Electronic check                  23   0.156609   
                             Mailed check                      69   0.063179   
               25-48         Mailed check                     121   0.066875   
Two year       0-12          Mailed check                      44   0.015334   
One year       13-24         Mailed check                      87   0.055533   
               49-72         Mailed check                      60   0.055729   
               13-24         Bank transfer (automatic)         45   0.081799   
                             Credit card (automatic)           42   0.089542   
               0-12          Credit card (automatic)           20   0.067982   
               13-24         Electronic check                  23   0.135782   
Two year       25-48         Electronic check                  27   0.081002   
               13-24         Mailed check                      58   0.014866   
               25-48         Credit card (automatic)           80   0.024415   
                             Bank transfer (automatic)         64   0.028167   
               0-12          Credit card (automatic)           14   0.014806   
               49-72         Mailed check                     177   0.013007   
               0-12          Bank transfer (automatic)          9   0.016044   
One year       0-12          Bank transfer (automatic)         12   0.061473   
Two year       25-48         Mailed check                     103   0.010884   
               13-24         Bank transfer (automatic)         12   0.016481   
                             Credit card (automatic)           18   0.012588   
                             Electronic check                   2   0.107572   
               0-12          Electronic check                   1   0.019001   
Month-to-month 72+           Bank transfer (automatic)  